In [2]:
SAMPLES_PATH = "/blue/rcstudents/smaley/pokegan/improved-diffusion/log_256/images"
NAMES_PATH = "/blue/rcstudents/smaley/pokegan/name-generation/pokemon-name-generator/output/generated_names.txt"

In [3]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

GRID_SIZE = 30  # 8x8 grid, limited to 8 * 8 = 64 images
OUTPUT_FILE = f"predictions_{timestr}.png"  # Output file name
OUTPUT_SIZE = (256,256,3)
INPUT_SIZE = (64,64,3)

In [4]:
TYPE_PREDICTION_MODEL_PATH = "/blue/rcstudents/smaley/pokegan/type-prediction/64_50_epoch.keras"

I used the following training parameters for my sampling.
Image sampling takes a long time, so this is all done ahead of time.
```bash
cd improved-diffusion
export OPENAI_LOGDIR="log_256" 
python scripts/image_sample.py --model_path /blue/rcstudents/smaley/pokegan/improved-diffusion/log_256/model_ckpts/ema_0.9999_990000.pt --image_size 256 --num_channels 128 --num_res_blocks 3 --diffusion_steps 4000 --noise_schedule linear --use_ddim True --num_samples 64
```

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

ImportError: Matplotlib requires numpy>=1.23; you have 1.19.5

In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpus

In [ ]:
from keras.models import load_model
model = load_model(TYPE_PREDICTION_MODEL_PATH)

In [ ]:
names = []
with open(NAMES_PATH, 'r') as file:
    names = file.readlines()
names = [line.strip() for line in names]

In [ ]:
pokemon = {}

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import csv
import random

# Variables
count = 0
# fig, axes = plt.subplots(GRID_SIZE, GRID_SIZE, figsize=(64, 64))
# axes = axes.flatten()

train = pd.read_csv('/blue/rcstudents/smaley/pokegan/type-prediction/image_tagging/train.csv', nrows=5) # set limit, delete "nrows" to undo
train.drop(['Name'],axis=1,inplace=True)

# Iterate through the dataset
for idx, file_name in enumerate(os.listdir(SAMPLES_PATH)):
    if count >= GRID_SIZE * GRID_SIZE:  # Limit to 64 images
        break
    
    img_path = os.path.join(SAMPLES_PATH, file_name)
    img = image.load_img(img_path, target_size=INPUT_SIZE)
    img_array = image.img_to_array(img) / 255.0  # Normalize
    classes = np.array(train.columns[2:])
    
    probs = model.predict(img_array.reshape(1,64,64,3))
    top_2 = np.argsort(probs[0])[:-3:-1]  # Get top 2 predictions
    if probs[0][top_2[1]] < (probs[0][top_2[0]] / 3): # monotype if confidence threshold is low, this should be rare
        top_2 = [top_2[0]]
    
    type_str = [f"{classes[i]} ({probs[0][i]:.2f})" for i in top_2]
    typing_to_prob = {}
    for i in range(len(classes)):
        typing_to_prob[classes[i]] = probs[0][i]
    
    # pick name at random 
    random_name = random.choice(names)
    names.remove(random_name)
    
    pokemon[file_name] = {"typing":typing_to_prob,"type_str":type_str,"name":random_name}
    count += 1

    
    # axes[idx].imshow(image.load_img(img_path,target_size=OUTPUT_SIZE))
    # axes[idx].axis('off')  # Turn off axis
    # axes[idx].set_title(f"P: {', '.join(type_str)}\nN: {random_name}", fontsize=24)
# plt.tight_layout()
# plt.savefig(f"results/{OUTPUT_FILE}")